<a href="https://colab.research.google.com/github/sidhu2690/CogniXis/blob/main/04_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [30]:
weights= torchvision.models.EfficientNet_B0_Weights.DEFAULT
Model=torchvision.models.efficientnet_b0(weights=weights)

In [31]:
pip install -q torchinfo

In [32]:
from torchinfo import summary

In [33]:
summary(
    model=Model,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"]
)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [1, 3, 224, 224]          [1, 1000]                 --                        True
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        True
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         --                        True
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         864                       True
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         64                        True
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─Sequential: 2-2  

In [34]:
for param in Model.features.parameters():
  param.requires_grad= False

In [35]:
summary(
    model=Model,
    input_size=(1, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [1, 3, 224, 224]          [1, 1000]                 --                        Partial
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        False
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         --                        False
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         (864)                     False
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         (64)                      False
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─Sequential

In [36]:
Model.classifier= nn.Sequential(
    nn.Dropout(p=0.2, inplace= True),
    nn.Linear(in_features=1280, out_features=3)
)

In [37]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [38]:
Model= Model.to(device)

In [39]:
!wget -q https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip -O pizza_steak_sushi.zip
!unzip -q pizza_steak_sushi.zip -d ./data/

replace ./data/test/steak/296375.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [42]:
train_dir = "/content/data/train"
test_dir = "/content/data/test"

In [43]:
data_transforms=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [44]:
train_data=datasets.ImageFolder(root=train_dir, transform= data_transforms)
test_data= datasets.ImageFolder(root=test_dir, transform=data_transforms)

In [46]:
train_loader= DataLoader(train_data, batch_size= 32, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 32, shuffle= False)

In [61]:
loss_fn=nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(params=Model.parameters(),
                           lr=0.1)

In [88]:
def training_loop(model, train_loader, test_loader, loss_fn, optimizer, epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    epoch_values = []
    train_loss_values = []
    test_loss_values = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        epoch_values.append(epoch)

        for X, y in train_loader:
            X, y = X.to(device), y.to(device)

            # Forward pass
            train_pred = model(X)
            loss = loss_fn(train_pred, y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate metrics
            train_loss += loss.item()
            train_correct += (train_pred.argmax(1) == y).sum().item()

        # Calculate epoch metrics
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        avg_train_loss = train_loss / len(train_loader)

        # Evaluate the model
        model.eval()
        test_loss = 0
        test_correct = 0

        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)

                test_pred = model(X)
                loss = loss_fn(test_pred, y)

                test_loss += loss.item()
                test_correct += (test_pred.argmax(1) == y).sum().item()

        # Calculate test metrics
        test_accuracy = test_correct / len(test_loader.dataset) * 100
        avg_test_loss = test_loss / len(test_loader)

        # Append losses for the epoch
        train_loss_values.append(avg_train_loss)
        test_loss_values.append(avg_test_loss)

        print(
            f"Epoch: {epoch+1}/{epochs} | "
            f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | "
            f"Test Loss: {avg_test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%"
        )

    return epoch_values, train_loss_values, test_loss_values
epochs = 15
epoch_values, train_loss_values, test_loss_values = training_loop(
    Model, train_loader, test_loader, loss_fn, optimizer, epochs
)


Epoch: 1/15 | Train Loss: 0.2891 | Train Accuracy: 96.44% | Test Loss: 0.3756 | Test Accuracy: 85.33%
Epoch: 2/15 | Train Loss: 0.1980 | Train Accuracy: 94.67% | Test Loss: 0.3436 | Test Accuracy: 86.67%
Epoch: 3/15 | Train Loss: 0.1251 | Train Accuracy: 95.11% | Test Loss: 0.2931 | Test Accuracy: 94.67%
Epoch: 4/15 | Train Loss: 0.1174 | Train Accuracy: 98.22% | Test Loss: 0.2843 | Test Accuracy: 93.33%
Epoch: 5/15 | Train Loss: 0.2684 | Train Accuracy: 97.78% | Test Loss: 0.5494 | Test Accuracy: 85.33%
Epoch: 6/15 | Train Loss: 0.3168 | Train Accuracy: 96.89% | Test Loss: 0.3516 | Test Accuracy: 85.33%
Epoch: 7/15 | Train Loss: 0.1806 | Train Accuracy: 96.00% | Test Loss: 0.3824 | Test Accuracy: 92.00%
Epoch: 8/15 | Train Loss: 0.0881 | Train Accuracy: 96.89% | Test Loss: 0.3297 | Test Accuracy: 94.67%
Epoch: 9/15 | Train Loss: 0.2669 | Train Accuracy: 97.78% | Test Loss: 0.4251 | Test Accuracy: 86.67%
Epoch: 10/15 | Train Loss: 0.0798 | Train Accuracy: 96.89% | Test Loss: 0.3364 | T